In [3]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.base import BaseEstimator
from sklearn.datasets import make_classification, make_regression, load_digits, load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from scipy.optimize import minimize
from sklearn.tree import DecisionTreeRegressor

In [4]:
class DecisionTree(object):
    ''' Реализация дерева решений для классификации и задачи регрессии'''
    
    def __init__(self, max_depth, min_size, estimator_type = 'classification'):
        self.max_depth = max_depth
        self.min_size = min_size
        if estimator_type == 'classification':
            self._criterion = self.gini
            self.add_node = self.node_classification
        if estimator_type == 'regression':
            self._criterion = self.mse
            self.add_node = self.node_regression
            
    #Вычисление индекса Джини
    def gini(self, y):
        p = [float(np.sum(y==val))/len(y) for val in np.unique(y)]
        return np.sum([p_i*(1-p_i) for p_i in p])

    #Вычисление среднеквадротичной ошибки
    def mse(self, y):
        if len(y) == 0:
            return 0
        return mean_squared_error(y, np.array([np.mean(y)]*len(y)))
    
    # Разбиение выборки по заданному значению 
    def division_set(self, index, value, dataset):
        left, right = [], []
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    
    # Выбор лучшего разбиения 
    def get_split(self, dataset):
        class_values = list(set(row[-1] for row in dataset))
        b_index, b_value, b_score, b_groups = 999, 999, 999, None
        for index in range(len(dataset[0])-1):
            for row in dataset:
                group_l, group_r = self.division_set(index, row[index], dataset)
                y_l = [x[-1] for x in group_l]
                y_r = [x[-1] for x in group_r]
                criterion = self._criterion(y_l)+self._criterion(y_r)
                if criterion < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], criterion, [group_l, group_r]
        return {'index':b_index, 'value':b_value, 'groups':b_groups}
    
    
    # Создание узла для классификации
    def node_classification(self, y):
        return np.unique(y)[np.argmax([np.sum(y==val) \
                                                      for val in np.unique(y)])]
    
    # Создание узла для регрессии 
    def node_regression(self, y):
        return np.mean(y)
      
    # Создание ветвления
    def _split(self, node, max_depth, min_size, depth):
        left, right = node['groups']
        y_l = [x[-1] for x in left]
        y_r = [x[-1] for x in right]
        del(node['groups'])
        if not left or not right:
            node['left'] = node['right'] = self.add_node(y_l + y_r)
            return
        if depth >= max_depth:
            node['left'], node['right'] = self.add_node(y_l), self.add_node(y_r)
            return
        if len(left) <= min_size:
            node['left'] = self.add_node(y_l)
        else:
            node['left'] = self.get_split(left)
            self._split(node['left'], max_depth, min_size, depth+1)
        if len(right) <= min_size:
            node['right'] = self.add_node(y_r)
        else:
            node['right'] = self.get_split(right)
            self._split(node['right'], max_depth, min_size, depth+1)
            

    #Создание прогноза
    def get_predict(self, node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.get_predict(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.get_predict(node['right'], row)
            else:
                return node['right']
   
    # Вывод дерева
    def _print_tree(self, node, depth=0, labels=''):
        if isinstance(node, dict):
            if labels == '':
                print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
            if labels != '':
                print('%s[%s < %.3f]' % ((depth*' ', (labels[node['index']+1]), node['value'])))
            self._print_tree(node['left'], depth = depth+1, labels = labels)
            self._print_tree(node['right'], depth = depth+1, labels = labels)
        else:
            print('%s[%s]' % ((depth*' ', node)))
    
    def print_tree(self, labels=''):
        self._print_tree(self.tree, labels=labels)
        
    def fit(self, X, y):
        y.shape = (len(y),1) 
        train = np.append(X, y, axis=1)
        root = self.get_split(train)
        self._split(root, self.max_depth, self.min_size, 1)
        self.tree = root
    
    def predict(self, X):
        predictions = []
        for row in X:
            predict = self.get_predict(self.tree, row)
            predictions.append(predict)
        return(predictions)

In [7]:
from sklearn.datasets import load_boston
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

tree = DecisionTree(max_depth = 2, min_size = 5, estimator_type='classification')
tree.fit(X_train, y_train)

print("Accuracy on training data: {0}".format(accuracy_score(y_train, tree.predict(X_train))))
print("Accuracy on test data: {0}".format(accuracy_score(y_test, tree.predict(X_test))))

labels = ['sepal length', 'sepal width', 'petal length', 'petal width']
tree.print_tree(labels)

(120, 4) (30, 4) (120,) (30,)
Accuracy on training data: 0.9583333333333334
Accuracy on test data: 0.9
[petal width < 3.000]
 [sepal width < 5.400]
  [0.0]
  [0.0]
 [petal width < 5.000]
  [1.0]
  [2.0]


In [8]:
boston = load_boston()
X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tree = DecisionTree(max_depth = 30, min_size = 1, estimator_type = 'regression')
tree.fit(X_train, y_train)

print("Mean squared error on train data: {0}".format(mean_squared_error(y_train, tree.predict(X_train))))
print("Mean squared error on test data {0}".format(mean_squared_error(y_test, tree.predict(X_test))))

Mean squared error on train data: 64.37139673758499
Mean squared error on test data 85.24491564653458


In [9]:
labels = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
tree.print_tree(labels)

[RAD < 1.174]
 [ZN < 8.267]
  [50.0]
  [50.0]
 [CHAS < 0.740]
  [50.0]
  [MEDV < 1.980]
   [50.0]
   [MEDV < 36.980]
    [MEDV < 2.470]
     [34.9]
     [MEDV < 2.870]
      [41.7]
      [MEDV < 2.880]
       [36.4]
       [MEDV < 2.940]
        [50.0]
        [ZN < 88.976]
         [ZN < 67.921]
          [MEDV < 2.960]
           [33.4]
           [MEDV < 2.980]
            [50.0]
            [MEDV < 3.010]
             [32.0]
             [MEDV < 3.110]
              [46.0]
              [MEDV < 3.130]
               [ZN < 0.022]
                [44.0]
                [42.3]
               [MEDV < 3.160]
                [37.6]
                [INDUS < 100.000]
                 [LSTAT < 2.600]
                  [13.4]
                  [RAD < 1.285]
                   [13.8]
                   [ZN < 0.006]
                    [22.155874673629246]
                    [22.155874673629246]
                 [31.6]
          [5.0]
         [10.4]
    [7.0]
